In [4]:
import pandas as pd
from gensim.models import Word2Vec

In [2]:
main_data = pd.read_csv('News_Final.csv')

In [3]:
main_data.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
0,99248.0,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,0.000000,-0.053300,-1,-1,-1
1,10423.0,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,0.208333,-0.156386,-1,-1,-1
2,18828.0,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-0.425210,0.139754,-1,-1,-1
3,27788.0,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,0.000000,0.026064,-1,-1,-1
4,27789.0,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,0.000000,0.141084,-1,-1,-1


* Word to vec 
* Clustering 
* Exploration of popularity
* Exploration of topics favored by sources
* GenSim NLP library

Step 1: Learning Word2Vec and doing some preliminary text analysis on the titles/headlines

Step 2: (Separate notebook) EDA

Contact Jeremie via Slack if any questions

In [12]:
article_titles = main_data.Title
len(article_titles)

93239

In [11]:
model = Word2Vec(article_titles)

93239

In [15]:
model.train(article_titles, total_examples = len(article_titles), epochs = 10)

(13272258, 53366780)

In [16]:
model.vector_size

100

In [19]:
for word in model.vocabulary:
    print(word)

TypeError: 'Word2VecVocab' object is not iterable

In [24]:
model.wv.most_similar(positive = 'economy')

KeyError: "word 'economy' not in vocabulary"

In [27]:
model['economy']

/Users/charlenechambliss/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


KeyError: "word 'economy' not in vocabulary"

In [25]:
main_data.Title[:20]

0      Obama Lays Wreath at Arlington National Cemetery
1           A Look at the Health of the Chinese Economy
2      Nouriel Roubini: Global Economy Not Back to 2008
3                             Finland GDP Expands In Q4
4     Tourism, govt spending buoys Thai economy in J...
5     Intellitec Solutions to Host 13th Annual Sprin...
6                                   Monday, 29 Feb 2016
7     Obama, stars pay a musical tribute to Ray Charles
8     Fire claims more than 100-year-old barn in Han...
9           Microsoft's new Windows 10 ad targets Apple
10    Microsoft Project Centennial seen with “univer...
11    Microsoft sneaks onto Android while Android sn...
12    Greek economy grows 0.1 pct q/q in fourth quarter
13    Big data and the Internet of Things to add £32...
14    HoloLens dev edition costs £2,000 with new Con...
15    Microsoft Word for Mac 2016 review: A subtle y...
16       Microsoft Band 2 gains appeal with lower price
17    Microsoft prepares to unleash Office 2016 